In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('ces_hourly_earnings_2022_2025.csv')

In [3]:
df.groupby("industry")["avg_hourly_earnings"].describe()
df["date"].min(), df["date"].max()


('2022-01-01', '2025-12-01')

In [5]:
!pip install -U pandas numpy sentence-transformers scikit-learn

     ------------------------------------- 493.7/493.7 KB 10.3 MB/s eta 0:00:00
     ---------------------------------------- 8.9/8.9 MB 11.4 MB/s eta 0:00:00
     --------------------------------------- 12.0/12.0 MB 11.3 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.3
    Uninstalling transformers-4.33.3:
      Successfully uninstalled transformers-4.33.3
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.6.1
    Uninstalling sentence-transformers-2.6.1:
   

You should consider upgrading via the 'C:\Users\ammsh\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
# Fix version compatibility - run this cell, then RESTART THE KERNEL before running the next cell
# The kernel restart is required because old module versions are cached in memory

# Downgrade NumPy to 1.x (TensorFlow was compiled against NumPy 1.x)
%pip install "numpy<2"

# Install compatible versions of sentence-transformers
%pip install sentence-transformers==2.2.2 transformers==4.35.0

print("\n" + "="*60)
print("IMPORTANT: Restart the kernel now before running the next cell!")
print("Kernel > Restart Kernel (or press 0,0 in command mode)")
print("="*60)

     --------------------------------------- 15.8/15.8 MB 11.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\ammsh\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\~umpy.libs\\libscipy_openblas64_-13e2df515630b4a41f92893938845698.dll'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\Users\ammsh\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Using cached transformers-4.35.0-py3-none-any.whl (7.9 MB)
     ---------------------------------------- 2.2/2.2 MB 10.8 MB/s eta 0:00:00
  Using cached huggingface_hub-0.17.3-py3-none-any.whl (295 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.35.1
    Uninstalling huggingface-hub-0.35.1:
      Successfully uninstalled huggingface-hub-0.35.1
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
Note: you may need to restart the kernel to use updated packages.

IMPORTANT: Restart the kernel now before running the next cell!
Kernel > Restart Kernel (or press 0,0 in command mode)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\ammsh\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\~-kenizers\\tokenizers.pyd'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\Users\ammsh\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
"""
Compute SOC-level AI similarity from O*NET text using embeddings + cosine similarity.

Inputs (download from O*NET Database, e.g., "O*NET Database 29.x"):
  Place these files in a folder, e.g. ./onet_db/
  Works best if you have at least ONE of:
    - Task Statements.txt
    - Skills.txt
    - Technology Skills.txt
    - Tools Used.txt
    - Work Activities.txt
    - Detailed Work Activities.txt
    - Knowledge.txt
    - Abilities.txt

Output:
  soc_ai_similarity.csv  (columns: onet_soc_code, soc_2018, ai_similarity, n_rows_used, text_len)

Install:
  pip install -U pandas numpy sentence-transformers scikit-learn
"""

from pathlib import Path
import re
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# -----------------------------
# Config
# -----------------------------
ONET_DIR = Path("onet_db")  # folder containing O*NET .txt tables
OUT_CSV = "soc_ai_similarity.csv"

MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

# AI anchor text: tune this (add your keywords/tools)
AI_ANCHOR_TEXT = """
artificial intelligence AI machine learning ML deep learning neural networks
large language models LLMs generative AI transformers embeddings prompt engineering
natural language processing NLP computer vision CV speech recognition
classification regression clustering forecasting anomaly detection recommendation systems
automation decision support model deployment MLOps
Python R SQL TensorFlow PyTorch scikit-learn XGBoost LightGBM
vector databases retrieval augmented generation RAG
"""

# Optional: keep only SOC codes that look valid (O*NET-SOC style)
SOC_PATTERN = re.compile(r"^\d{2}-\d{4}\.\d{2}$|^\d{2}-\d{4}$")

# -----------------------------
# Helpers
# -----------------------------
def read_onet_table(path: Path) -> pd.DataFrame:
    """
    O*NET .txt tables are typically tab-delimited. We'll try tab first, then comma.
    """
    for sep in ["\t", ","]:
        try:
            df = pd.read_csv(path, sep=sep, dtype=str, engine="python")
            if df.shape[1] >= 2:
                df.columns = [c.strip() for c in df.columns]
                return df
        except Exception:
            pass
    raise RuntimeError(f"Could not parse {path.name} as tab or csv.")

def find_soc_col(df: pd.DataFrame) -> str | None:
    """
    SOC code is usually in 'O*NET-SOC Code'. Sometimes 'ONET-SOC Code' or similar.
    """
    candidates = [
        "O*NET-SOC Code",
        "ONET-SOC Code",
        "O_NET-SOC Code",
        "onet_soc_code",
        "SOC Code",
        "SOC",
    ]
    for c in candidates:
        if c in df.columns:
            return c
    # fuzzy fallback
    for c in df.columns:
        if "SOC" in c.upper() and "CODE" in c.upper():
            return c
    return None

def pick_text_cols(df: pd.DataFrame, table_name: str) -> list[str]:
    """
    Choose columns that contain the actual descriptive text for each table.
    """
    preferred = [
        # Common text fields across O*NET tables
        "Task",
        "Task Statement",
        "Task Statement (Text)",
        "Task Description",
        "Description",
        "Element Name",
        "Work Activity",
        "Detailed Work Activity",
        "Technology Skill",
        "Tool",
        "Commodity Title",
        "Title",
        "Knowledge",
        "Skill",
        "Ability",
        "Example",
        "Sample of Reported Titles",
    ]
    chosen = [c for c in preferred if c in df.columns]
    if chosen:
        return chosen

    # fallback: pick object-like columns excluding obvious ids
    drop_like = {"scale id", "element id", "datavalue", "n", "standard error", "domain source"}
    cols = []
    for c in df.columns:
        cl = c.strip().lower()
        if "code" in cl or "id" in cl:
            continue
        if any(k in cl for k in drop_like):
            continue
        cols.append(c)
    # keep a reasonable number
    return cols[:6]

def normalize_soc_to_2018(onet_soc: str) -> str:
    """
    Strip the .xx extension so '15-1252.00' -> '15-1252' (SOC 2018-like 6-digit occupation).
    """
    if onet_soc is None:
        return None
    return str(onet_soc).split(".")[0].strip()

def clean_text(s: str) -> str:
    s = re.sub(r"\s+", " ", str(s)).strip()
    return s

# -----------------------------
# Load O*NET tables and build SOC text corpus
# -----------------------------
TABLE_FILES = [
    "Task Statements.txt",
    "Skills.txt",
    "Technology Skills.txt",
    "Tools Used.txt",
    "Work Activities.txt",
    "Detailed Work Activities.txt",
    "Knowledge.txt",
    "Abilities.txt",
]

rows = []

for fname in TABLE_FILES:
    fpath = ONET_DIR / fname
    if not fpath.exists():
        continue

    df = read_onet_table(fpath)
    soc_col = find_soc_col(df)
    if soc_col is None:
        continue

    text_cols = pick_text_cols(df, fname)

    # keep only needed cols
    use = df[[soc_col] + text_cols].copy()
    use = use.rename(columns={soc_col: "onet_soc_code"})

    # concat text cols per row
    use["row_text"] = use[text_cols].astype(str).agg(" | ".join, axis=1).map(clean_text)

    # filter out empty row_text
    use = use[use["row_text"].str.len() > 0]

    # optional SOC format filter
    use["onet_soc_code"] = use["onet_soc_code"].astype(str).str.strip()
    use = use[use["onet_soc_code"].apply(lambda x: bool(SOC_PATTERN.match(x)) if isinstance(x, str) else False)]

    use["source_table"] = fname
    rows.append(use[["onet_soc_code", "row_text", "source_table"]])

if not rows:
    raise RuntimeError(
        f"No usable O*NET tables found in {ONET_DIR.resolve()}.\n"
        f"Put O*NET Database .txt files there (e.g., Task Statements.txt, Skills.txt, Technology Skills.txt)."
    )

onet_long = pd.concat(rows, ignore_index=True)

# aggregate all text per onet_soc_code
soc_docs = (
    onet_long.groupby("onet_soc_code", as_index=False)["row_text"]
    .apply(lambda x: "\n".join(x.tolist()))
    .rename(columns={"row_text": "doc"})
)

soc_docs["soc_2018"] = soc_docs["onet_soc_code"].map(normalize_soc_to_2018)

# Diagnostics
soc_docs["text_len"] = soc_docs["doc"].str.len()

# -----------------------------
# Embed + cosine similarity
# -----------------------------
model = SentenceTransformer(MODEL_NAME)

anchor_emb = model.encode([AI_ANCHOR_TEXT], normalize_embeddings=True)
docs_emb = model.encode(soc_docs["doc"].tolist(), normalize_embeddings=True, show_progress_bar=True)

soc_docs["ai_similarity"] = cosine_similarity(docs_emb, anchor_emb).ravel()

# Optional: add how many underlying rows contributed (across all tables)
counts = onet_long.groupby("onet_soc_code").size().reset_index(name="n_rows_used")
soc_docs = soc_docs.merge(counts, on="onet_soc_code", how="left")

# keep tidy output
out = soc_docs[["onet_soc_code", "soc_2018", "ai_similarity", "n_rows_used", "text_len"]].sort_values(
    "ai_similarity", ascending=False
)

out.to_csv(OUT_CSV, index=False)
print(f"Wrote {OUT_CSV} with {len(out):,} SOC rows")
print(out.head(10))


c:\Users\ammsh\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\ammsh\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\ammsh\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _r

: 


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\ammsh\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\ammsh\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\ammsh\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\ammsh\AppData\Local\Programs\Python\Python310\lib\site-packages\trait

: 


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\ammsh\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\ammsh\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\ammsh\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\ammsh\AppData\Local\Programs\Python\Python310\lib\site-packages\trait

: 

: 

: 